### Imports

In [7]:
%pylab inline 
import os
import nibabel as nib
import numpy as np
from glob import glob
from scipy.ndimage import label

Populating the interactive namespace from numpy and matplotlib


##### Load the text file with mse ids

In [8]:
subjects = np.genfromtxt("pbr_baselines.txt", dtype=str)
print subjects[:5]

['mse1328' 'mse1334' 'mse1337' 'mse1343' 'mse1349']


##### define some paths to data

In [9]:
base_dir = "/data/henry7/PBR/subjects/"
template = "{base_dir}/{mseid}/lesions_manual/*/alignment_lesions.nii.gz"

## figure out a way to fill the template and glob for the file you need

In [10]:
lesion_file = #fill the template with the first subject, subjects[0]

SyntaxError: invalid syntax (<ipython-input-10-08377618c11f>, line 1)

## Load an image

In [ ]:
img = nib.load(lesion_file)
data, affine = nib.get_data(), nib.get_affine()

## Number each lesion 

In [ ]:
labels, nlabels = label(data==1)

## Get x,y,z coordinates for each lesion

In [ ]:
np.nonzero

## Tranform coordinates to world space

In [ ]:
np.dot([x,y,z,1], aff) #order might be switched 

In [11]:
subjects = np.genfromtxt("pbr_baselines.txt", dtype=str)
print subjects[:5]

['mse1328' 'mse1334' 'mse1337' 'mse1343' 'mse1349']


In [12]:
%pylab inline 
import os
import nibabel as nib
import numpy as np
from glob import glob
from scipy.ndimage import label

Populating the interactive namespace from numpy and matplotlib


In [13]:
subjects = np.genfromtxt("pbr_baselines.txt", dtype=str)
print subjects[:5]

['mse1328' 'mse1334' 'mse1337' 'mse1343' 'mse1349']


In [14]:
import glob
import nibabel as nib

In [15]:
ms1328 = "/data/henry7/PBR/subjects/mse1328/lesions_manual/ms56-mse1328-002-AX_T1_3D_IRSPGR/alignment_lesions.nii.gz"

In [16]:
for lesion_file_1328 in glob.glob("/data/henry7/PBR/subjects/mse1328/lesions_manual/*/alignment_lesions.nii.gz"):
    print lesion_file_1328

/data/henry7/PBR/subjects/mse1328/lesions_manual/ms56-mse1328-002-AX_T1_3D_IRSPGR/alignment_lesions.nii.gz


In [17]:
img1328 = nib.load(lesion_file_1328)
img1328.dataobj

In [18]:
data1328 = img1328.get_data()
data1328.shape


(256, 256, 150)

In [19]:
affine1328 = img1328.get_affine()
print affine1328

[[  -0.9375        0.            0.          123.6289978 ]
 [   0.            0.9375        0.          -93.36050415]
 [   0.            0.            1.          -44.42710114]
 [   0.            0.            0.            1.        ]]


In [20]:
labels, nlabels = label(data1328==[1])

In [21]:
nlabels

11

In [22]:
n = np.unique(labels)
print n

[ 0  1  2  3  4  5  6  7  8  9 10 11]


In [23]:
np.bincount(labels.ravel())

array([9829648,      69,     105,      54,      29,      69,      61,
            36,      47,     125,     111,      46])

In [37]:
def get_lesion_coord():
    all_lesion = []
    for count in range(1, nlabels+1):
        cur_lesion = []
        x,y,z = np.nonzero(labels==count)
        for count2 in range(len(x)):
            cur_coordinates = [x[count2],y[count2],z[count2]]
            cur_lesion.append(cur_coordinates)
        all_lesion.append(cur_lesion)
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_lesion[count]
    #    print ""
    return all_lesion


In [54]:
def average_les(coordinates):
    averages_all = []
    for count in range(len(coordinates)):
        averages = []
        sumx=0;sumy=0;sumz=0
        for count2 in range(len(coordinates[count])):
            sumx += coordinates[count][count2][0]
            sumy += coordinates[count][count2][1]
            sumz += coordinates[count][count2][2]
        average_x = sumx / len(coordinates[count])
        average_y = sumy / len(coordinates[count])
        average_z = sumz / len(coordinates[count])
        averages = [average_x, average_y, average_z]
        averages_all.append(averages)
        #print count, averages
    return averages_all

average_lesions = average_les(get_lesion_coord())
print average_lesions

[[78, 105, 89], [86, 126, 82], [91, 93, 58], [90, 98, 50], [99, 82, 55], [102, 74, 61], [113, 77, 82], [146, 102, 89], [148, 148, 89], [150, 115, 101], [151, 106, 92]]


In [83]:
realspace_coordinates = np.dot(affine1328, [average_lesions[0][0],
                 average_lesions[0][1], 
                 average_lesions[0][2],
                 1])

In [84]:
average_lesions[0] 

[78, 105, 89]

In [85]:
realspace_coordinates

array([ 50.5039978 ,   5.07699585,  44.57289886,   1.        ])

In [86]:
from numpy.linalg import inv

In [78]:
inverse_affine_1328 = np.linalg.inv(affine_fs)
print inverse_affine_1328

[[  -1.06666667   -0.           -0.          131.87093099]
 [   0.            1.06666667    0.           99.58453776]
 [   0.            0.            1.           44.42710114]
 [   0.            0.            0.            1.        ]]


In [88]:
freesurfer_matrix = np.dot(affine_inverse1328, [realspace_coordinates[0],
                 realspace_coordinates[1], 
                 realspace_coordinates[2],
                 1])
print freesurfer_matrix

[  78.  105.   89.    1.]
